# Advanced Object-Oriented Programming: Metaclasses and Class Decorators

A `class` is a "factory for objects (instances)"

What is a "factory for classes?"

In [1]:
class MyClass1:
    pass

In [2]:
obj = MyClass1()
type(obj)

__main__.MyClass1

In [3]:
type(MyClass1)

type

In [4]:
isinstance(obj, MyClass1)

True

In [5]:
isinstance(MyClass1, type)

True

### Building classes with the `type` function

In [6]:
MyClass16 = type('MyClass16', (MyClass1,), {})

In [7]:
MyClass16

__main__.MyClass16

In [8]:
isinstance(MyClass16, type)

True

In [9]:
MyClass16.mro()

[__main__.MyClass16, __main__.MyClass1, object]

In [10]:
myobj16 = MyClass16()

In [11]:
isinstance(myobj16, MyClass16)

True

In [12]:
MyClass16 = type('MyClass16', (MyClass1,), {
    'training_class_attribute': 42
})

In [13]:
MyClass16.training_class_attribute

42

In [14]:
class MyClass16(MyClass1):
    training_class_attribute = 42

In [15]:
MyClass16.training_class_attribute

42

In [16]:
MyClass16.__dict__

mappingproxy({'__module__': '__main__',
              'training_class_attribute': 42,
              '__doc__': None})

In [17]:
def instance_initializer(self):
    '''This will become __init__ below...'''
    print('Creating an instance')
    self.training_instance_attribute = 31337
    
MyClass17 = type('MyClass17', (MyClass1,), {
    'training_class_attribute': 42,
    '__init__': instance_initializer,
})

In [18]:
myobj = MyClass17()

Creating an instance


In [19]:
myobj.training_instance_attribute

31337

In [20]:
myobj.training_class_attribute

42

In [21]:
MyClass17.__dict__

mappingproxy({'training_class_attribute': 42,
              '__init__': <function __main__.instance_initializer(self)>,
              '__module__': '__main__',
              '__doc__': None})

In [22]:
class MyClass17(MyClass1):
    training_class_attribute = 42
    
    def __init__(self):
        print('Creating an instance')
        self.training_instance_attribute = 31337

In [23]:
MyClass17.__dict__

mappingproxy({'__module__': '__main__',
              'training_class_attribute': 42,
              '__init__': <function __main__.MyClass17.__init__(self)>,
              '__doc__': None})

In [24]:
myobj.__dict__

{'training_instance_attribute': 31337}

# Introducing Metaclasses

metaclass is to class as class is to instance

In [25]:
%%python2
def meta(name, bases, dct):
    print "Calling the metaclass"
    dct['foo'] = 'bar'
    return type(name, bases, dct)

class MyClass2(object):
    __metaclass__ = meta

print MyClass2.foo

Calling the metaclass
bar


In [26]:
def cabbage(name, bases, dct):
    print('Inside meta, ', name, bases, dct)
    return type(name, bases, dct)

class MyClass2(metaclass=cabbage):
    a = 'foo'
    def __init__(self):
        pass


Inside meta,  MyClass2 () {'__module__': '__main__', '__qualname__': 'MyClass2', 'a': 'foo', '__init__': <function MyClass2.__init__ at 0x7f1c99727b80>}


## Why oh why??

In [27]:
!pip install marshmallow
import marshmallow as mm

Looking in links: /home/rick446/src/wheelhouse
You should consider upgrading via the '/home/rick446/.virtualenvs/classes/bin/python -m pip install --upgrade pip' command.


```json
{
    "first": "Rick",
    "last": "Copeland",
    "email": "rick@arborian.com"
}
```

In [28]:
class UserSchema(mm.Schema):
    first = mm.fields.Str(missing='Gutzon')
    last = mm.fields.Str(missing='Borglum')
    email = mm.fields.Str(required=True)

In [30]:
f = mm.fields.Str(missing='Gutzon')

In [31]:
f.name

```python
# NOT valid marshmallow
UserSchema = make_schema(
    fields=[
        Str('first', missing='Gutzon'),
        ...
    ]
)

UserSchema = make_schema(
    first=Str(missing='Gutzon'),
    ...
)
```

In [32]:
sch = UserSchema()
sch.load({'email': 'rick@arborian.com'})

{'last': 'Borglum', 'first': 'Gutzon', 'email': 'rick@arborian.com'}

In [33]:
mm.schema.Schema??

In [34]:
mm.schema.SchemaMeta??

### Perverse metaclass example

In [35]:
def make_weird_dict(name, bases, dct):
    return {
        name: value 
        for name, value in dct.items() 
        if not name.startswith('_')
    }

class Instructor(metaclass=make_weird_dict):
    name = 'Rick Copeland'
    city = 'Atlanta'

In [36]:
Instructor

{'name': 'Rick Copeland', 'city': 'Atlanta'}

In [37]:
type(Instructor)

dict

### </Perverse metaclass example>

# Introducing `__new__`

In [38]:
class NonMeta:
    def __new__(cls, *args):
        print('Calling __new__ with', args)
        self = super().__new__(cls)
        self.extra_attribute = 'Victory!'
        return self
    
    def __init__(self, a, b):
        print('Calling __init__', a, b)
        self.a, self.b = a, b


In [39]:
inst = NonMeta(1,2)

Calling __new__ with (1, 2)
Calling __init__ 1 2


In [40]:
inst.extra_attribute

'Victory!'

In [41]:
inst.a, inst.b

(1, 2)

In [42]:
# type(a, b, c) ==> type.__new__(type, a, b, c)
class WithAMeta(type):
    def __new__(meta, name, bases, dct):
        print('Create new class', name, 'with WithAMeta metaclass')
        dct['a'] = 5
        return super().__new__(meta, name, bases, dct)
        # return type.__new__(meta, name, bases, dct)
    

In [43]:
class MyClass3(metaclass=WithAMeta):
    pass

Create new class MyClass3 with WithAMeta metaclass


In [44]:
MyClass3.a

5

In [45]:
type(MyClass3)

__main__.WithAMeta

In [46]:
MyClass3 = WithAMeta('MyClass3', (), {})

Create new class MyClass3 with WithAMeta metaclass


In [47]:
MyClass3.a

5

In [48]:
type(MyClass3)

__main__.WithAMeta

In [49]:
type(WithAMeta)

type

In [50]:
isinstance(WithAMeta, type)

True

In [51]:
issubclass(WithAMeta, type)

True

"Proper" metaclasses are inherited by subclasses; "function" metaclasses are not

In [52]:
def metafun(name, bases, dct):
    print('Call metafun')
    return type(name, bases, dct)

In [53]:
class MetaCls(type):
    def __new__(meta, name,bases, dct):
        print('Call metacls')
        return super().__new__(meta, name, bases, dct)

In [54]:
class FunCls(metaclass=metafun):
    pass

class ClsCls(metaclass=MetaCls):
    pass

Call metafun
Call metacls


In [55]:
class FunCls2(FunCls): pass

In [56]:
type(FunCls)

type

In [57]:
class ClsCls2(ClsCls): pass

Call metacls


In [58]:
type(ClsCls)

__main__.MetaCls

### Metaclass trick: instance level property declaration

In [ ]:
from copy import deepcopy

class PropertyMeta(type):
    def __new__(meta, name, bases, dct):
        print('Create class', name)
        public_attributes = {
            key: value 
            for key, value in dct.items()
            if not callable(value)   # no methods
            if not key.startswith('_')   # no private attrs
        }
        dct['_public_attributes'] = public_attributes
        return super().__new__(meta, name, bases, dct)
    
    

In [ ]:
class PublicBase(metaclass=PropertyMeta):
    def __init__(self):
        for key, value in self._public_attributes.items():
            setattr(self, key, deepcopy(value))

In [ ]:
class PublicClass(PublicBase):
    a = 5
    b = 6
    c = [7, 8, 9]
    
    def amethod(self):
        print('Calling amethod!')

In [ ]:
PublicClass._public_attributes

In [ ]:
pc = PublicClass()

In [ ]:
pc.__dict__

In [ ]:
PublicClass.__dict__

In [ ]:
pc.c.append(10)

In [ ]:
pc.c

In [ ]:
PublicClass.c

(a better way is to use dataclasses from the standard lib)

### Metaclass use case: collecting properties ('declarative programming')

In [59]:
class ImportantField:
    def __init__(self, value, name=None):
        self.value = value
        self.name = name
        
    def __repr__(self):
        return f'<Important {self.name}: {self.value}>'

In [60]:
class DeclarativeMeta(type):
    def __new__(meta, name, bases, dct):
        print(f'DeclarativeMeta({meta!r}, {name!r}, {bases!r}, {dct!r})')
        important = []
        for k, v in dct.items():
            if isinstance(v, ImportantField):
                if v.name is None:
                    v.name = k
                important.append(v)
        dct['_important'] = tuple(important)
        return super().__new__(meta, name, bases, dct)

In [61]:
class DeclarativeBase(metaclass=DeclarativeMeta):
    """Use regular inheritance to get the metaclass 'for free'"""

    def __repr__(self):
        l = ['<{}'.format(self.__class__.__name__)]
        for fld in self._important:
            l.append(' {}={}'.format(fld.name, fld.value))
        l.append('>')
        return ''.join(l)
    

DeclarativeMeta(<class '__main__.DeclarativeMeta'>, 'DeclarativeBase', (), {'__module__': '__main__', '__qualname__': 'DeclarativeBase', '__doc__': "Use regular inheritance to get the metaclass 'for free'", '__repr__': <function DeclarativeBase.__repr__ at 0x7f1c9a2294c0>})


In [62]:
DeclarativeBase._important

()

In [63]:
# Here is the "application-level" code you'd write
    
class MyClass4(DeclarativeBase):
    a = ImportantField(1)
    b = ImportantField(2)
    c = ImportantField(3, name='cfield')

DeclarativeMeta(<class '__main__.DeclarativeMeta'>, 'MyClass4', (<class '__main__.DeclarativeBase'>,), {'__module__': '__main__', '__qualname__': 'MyClass4', 'a': <Important None: 1>, 'b': <Important None: 2>, 'c': <Important cfield: 3>})


In [64]:
MyClass4._important

(<Important a: 1>, <Important b: 2>, <Important cfield: 3>)

In [65]:
MyClass4()

<MyClass4 a=1 b=2 cfield=3>

In [66]:
MyClass4.c

<Important cfield: 3>

In [67]:
MyClass4.a

<Important a: 1>

In [68]:
obj = MyClass4()
obj.a

<Important a: 1>

In [69]:
obj.c

<Important cfield: 3>

In [70]:
obj

<MyClass4 a=1 b=2 cfield=3>

### Aside: metaclass inheritance

If ClassSub with metaclass MetaClassB extends ClassSuper with MetaClassA, then
MetaClassB must be a subclass of MetaClassA:

In [74]:
# Metaclasses
class MetaClassA(type):
    pass

class MetaClassB(MetaClassA):
    pass

class MetaClassC(type):
    pass

# "Regular" classes
class ClassSuper(metaclass=MetaClassA):
    pass

class ClassSub(ClassSuper, metaclass=MetaClassC):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

### Metaclass use case: class registries

In [75]:
class RegistryMeta(type):
    _registry = {}
    
    def __new__(meta, name, bases, dct):
        cls = super().__new__(meta, name, bases, dct)
        meta._registry[name] = cls
        return cls
    
    def lookup_meta(cls, name):
        return cls._registry[name]

class RegistryBase(metaclass=RegistryMeta):
    
    @classmethod
    def lookup(cls, name):
        return cls._registry[name]

In [76]:
RegistryMeta._registry

{'RegistryBase': __main__.RegistryBase}

In [77]:
class Registered1(RegistryBase):
    a = ImportantField('Registered2')

class Registered2(RegistryBase):
    pass
    
class Registered3(metaclass=RegistryMeta):
    pass

In [78]:
RegistryMeta._registry

{'RegistryBase': __main__.RegistryBase,
 'Registered1': __main__.Registered1,
 'Registered2': __main__.Registered2,
 'Registered3': __main__.Registered3}

In [79]:
RegistryBase.lookup('Registered1')

__main__.Registered1

In [80]:
RegistryBase.lookup('Registered2')

__main__.Registered2

In [81]:
RegistryBase.lookup_meta('Registered1')

__main__.Registered1

In [82]:
RegistryBase.lookup_meta('Registered2')

__main__.Registered2

In [83]:
RegistryBase.lookup_meta('Registered3')

__main__.Registered3

In [84]:
class Employee(mm.Schema):
    name = mm.fields.Str()
    supervisor = mm.fields.Nested('Employee')

### Metaclass trick: class operators

In [85]:
class RegistryMeta(type):
    _registry = {}
    
    def __new__(meta, name, bases, dct):
        cls = super().__new__(meta, name, bases, dct)
        meta._registry[name] = cls
        return cls
    
    def __getitem__(cls, name):
        """Implements cls[name]"""
        return cls._registry[name]
    
    def mclass_method(cls):
        """cls is an instance of RegistyMeta, which is a class"""
        pass
    
class RegistryBase(object, metaclass=RegistryMeta):
    pass
  
#     sadly, this does not work...
#     @classmethod
#     def __getitem__(cls, name):
#         return cls._registry[name]
    
class Registered1(RegistryBase):
    def regular_method(self):
        '''self is an instance of Registered1'''
        pass
    
    @classmethod
    def class_method(cls):
        '''cls is Registered1'''
        pass

class Registered2(RegistryBase):
    pass

In [86]:
RegistryBase['Registered1']

__main__.Registered1

In [87]:
RegistryBase['Registered2']

__main__.Registered2

Trick: inheritance via `+`

In [88]:
class HierMeta(type):
    
    def __add__(cls, other):
        return type(
            f'<AnonCls>({cls.__name__}, {other.__name__})', 
            (cls, other), 
            {}
        )

class HierBase(metaclass=HierMeta): pass

In [89]:
class R1(HierBase): pass
class R2(HierBase): pass

In [90]:
R3 = R1 + R2

In [91]:
R3

__main__.<AnonCls>(R1, R2)

In [92]:
R3.mro()

[__main__.<AnonCls>(R1, R2),
 __main__.R1,
 __main__.R2,
 __main__.HierBase,
 object]

In [93]:
class Animal(HierBase): pass
class Biped(Animal): pass
class Quadruped(Animal): pass
class Monkey(Biped): pass
class Dog(Quadruped): pass
MonkeyDog = Monkey + Dog

In [94]:
MonkeyDog

__main__.<AnonCls>(Monkey, Dog)

In [95]:
MonkeyDog.mro()

[__main__.<AnonCls>(Monkey, Dog),
 __main__.Monkey,
 __main__.Biped,
 __main__.Dog,
 __main__.Quadruped,
 __main__.Animal,
 __main__.HierBase,
 object]

In [96]:
class ChangeNameMeta(type):
    def __new__(meta, name, bases, dct):
        dct['__qualname__'] = f'Changed{dct["__qualname__"]}'
        return super().__new__(meta, f'Changed{name}', bases, dct)

In [97]:
class MyName(metaclass=ChangeNameMeta): pass

In [98]:
MyName.__name__

'ChangedMyName'

In [99]:
MyName

__main__.ChangedMyName

In [100]:
MyName()

Marshmallow uses registries, as well

In [101]:
class TreeSchema(mm.Schema):
    value = mm.fields.Str()
    left = mm.fields.Nested('TreeSchema', missing=None)
    right = mm.fields.Nested('TreeSchema', missing=None)

In [102]:
schema = TreeSchema()

In [103]:
schema.load({'value': 'root', 'left': {
    'value': 'left'
}})

{'left': {'left': None, 'value': 'left', 'right': None},
 'value': 'root',
 'right': None}

### Class decorators: cheap substitute for metaclasses

Recall that

```python
@foo
def func(...):
    ...
```

really means

```python
def func(...):
    ...
func = foo(func)
```

In [104]:
def decorator_factory(a, b):
    def decorator(function):
        print(f'Decorating {function} with ({a}, {b}): {id(decorator)}')
        def wrapper(*args, **kwargs):
            print('Calling decorated function', a, b)
            return function(*args, **kwargs)
        return wrapper
    print(f'Creating decorator with args({a}, {b}): {id(decorator)}')
    return decorator

In [105]:
deco = decorator_factory(1,2)

Creating decorator with args(1, 2): 139760820748496


In [106]:
@deco
def my_cool_function(c, d, e):
    print('cool: ', c, d, e)

Decorating <function my_cool_function at 0x7f1c9a130280> with (1, 2): 139760820748496


In [107]:
my_cool_function

<function __main__.decorator_factory.<locals>.decorator.<locals>.wrapper(*args, **kwargs)>

In [108]:
my_cool_function(10, 20, e=256)

Calling decorated function 1 2
cool:  10 20 256


In [109]:
@decorator_factory(1,2)
def my_cool_function(c, d, e):
    print('cool: ', c, d, e)

Creating decorator with args(1, 2): 139760820750656
Decorating <function my_cool_function at 0x7f1c9a130700> with (1, 2): 139760820750656


In [110]:
my_cool_function(5,6,7)

Calling decorated function 1 2
cool:  5 6 7


```python
@locking(the_vault)
def transfer_money(a, b, amount):
    pass
```

We can do the same with classes:

```python
@foo
class Bar:
    ...
```

means

```python
class Bar:
    ...
Bar = foo(Bar)
```

In [111]:
def deco(cls):
    print(f'Calling deco({cls})')
    return cls

def meta(name, bases, dct):
    print(f'Calling meta({name})')
    return type(name, bases, dct)

@deco
class MyClass(metaclass=meta):
    print('In class definition')
    
print('Class body, metaclass, and decorator have all run by now')

In class definition
Calling meta(MyClass)
Calling deco(<class '__main__.MyClass'>)
Class body, metaclass, and decorator have all run by now


In [112]:
class Registry:
    
    def __init__(self):
        self._registry = {}

    def register(self, cls):
        self._registry[cls.__name__] = cls
        return cls

    def __getitem__(self, name):
        return self._registry[name]
    
r = Registry()

In [113]:
# "Application" code
@r.register
class Registered1:
    pass

#Registered1 = r.register(Registered1)

@r.register
class Registered2():
    pass

In [114]:
r._registry

{'Registered1': __main__.Registered1, 'Registered2': __main__.Registered2}

In [115]:
r['Registered1']

__main__.Registered1

In [116]:
r['Registered2']

__main__.Registered2

Real-world example: [Flask-RESTPlus][rest+]

[rest+]: https://flask-restplus.readthedocs.io/en/stable/quickstart.html#a-minimal-api

### ImportantField example

In [117]:
class ImportantField:
    def __init__(self, value, name=None):
        self.value = value
        self.name = name
        
    def __repr__(self):
        return f'<Important {self.name}: {self.value}>'
        
        
def declarative(cls):
    print('Running declarative() decorator')
    _important = []
    for name in dir(cls):
        value = getattr(cls, name)
        if isinstance(value, ImportantField):
            if value.name is None:
                value.name = name
            _important.append(value)
    cls._important = _important
    return cls
    
@declarative
class MyClass:
    def __repr__(self):
        return f"<MyClass {' '.join(repr(i) for i in self._important)} >"
    
    a = ImportantField(1)
    b = ImportantField(2)
    c = ImportantField(3, 'cfield')

    

Running declarative() decorator


In [118]:
obj = MyClass()

In [119]:
obj

<MyClass <Important a: 1> <Important b: 2> <Important cfield: 3> >

In [120]:
MyClass._important

[<Important a: 1>, <Important b: 2>, <Important cfield: 3>]

Class decorators are _not_ inherited:

In [125]:
class OtherClass(MyClass):
    d = ImportantField(5)

In [126]:
OtherClass()

<MyClass <Important a: 1> <Important b: 2> <Important cfield: 3> >

Example from Barin: 
    
```python
@b.cmap(class_collection)
class MyClass(library.MySuperClass):
    a = SomeThing()
    b = SomeThing()
```

Another real-world example: [DataClasses][dataclasses]

[dataclasses]: https://docs.python.org/3/library/dataclasses.html

## Metaclass semantics

bar = Foo()

- if Foo is a class, bar is an instance
- if Foo is a metaclass, bar is a class

- `type` is a metaclass
- `object` is class
- `'foo'` is an instance (of the class `str`, which has metaclass `type`)

In [127]:
type('foo')

str

In [128]:
type(str)

type

But how does this work?...

In [129]:
class Duck:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        print("Calling name's getter")
        return self._name
    
    @name.setter
    def name(self, value):
        print("Calling name's setter")
        if value.lower() in ('ed', 'bugs', 'porky'):
            raise ValueError('Inappropriate name')
        self._name = value
    #name = name.setter(name)

In [130]:
d = Duck('Donald')

In [131]:
d.name

Calling name's getter


'Donald'

In [132]:
d.name = 'Lewey'

Calling name's setter


In [133]:
d.name = 'ed'

Calling name's setter


ValueError: Inappropriate name

In [134]:
Duck.name.__delete__

<method-wrapper '__delete__' of property object at 0x7f1c9a0018b0>

# Descriptors

... or "ever wonder how `@property`, `@classmethod`, and `@staticmethod` work?"

**Descriptors** are object which contain one or more of the following magic methods, and which occur in a class body:

- `__get__(self, inst, type)` - called when the descriptor attribute is looked up (e.g. `getattr()`)
- `__set__(self, inst, value)` - called when the descriptor attribute is set (e.g. `setattr()`)
- `__delete__(self, inst)` - called when the descriptor attribute is deleted (e.g. `delattr()` or `del inst.attr`)
- `__set_name__(self, cls, name)` - called to notify the descriptor of its name within the class

In [135]:
class MyDesc:
    
    def __get__(self, inst, cls):
        # default behavior is something like return inst.__dict__[my_own_name]
        print(f'Calling __get__({self}, {inst}, {cls})')
        if inst is None:
            return self

    def __set__(self, inst, value):
        print(f'Calling __set__({self}, {inst}, {value})')

    def __delete__(self, inst):
        print(f'Calling __del__({self}, {inst})')
        
    def __set_name__(self, cls, name):
        print(f'Setting name of the descriptor {self} in class {cls} to {name}')

class MyClass:
    a = MyDesc()       # a.__set_name__ is called at class creation time
    b = a
    c = a
    
    def __repr__(self):
        return '<Instance of MyClass>'


Setting name of the descriptor <__main__.MyDesc object at 0x7f1c9a1b5220> in class <class '__main__.MyClass'> to a
Setting name of the descriptor <__main__.MyDesc object at 0x7f1c9a1b5220> in class <class '__main__.MyClass'> to b
Setting name of the descriptor <__main__.MyDesc object at 0x7f1c9a1b5220> in class <class '__main__.MyClass'> to c


```python
# Psuedo-code for what's happening at class creation time
for key, value in dct.items():
    if hasattr(value, '__set_name__'):
        value.__set_name__(key)
```

In [136]:
myobj = MyClass()

In [137]:
MyClass.a   # invokes a.__get__(None, MyClass)

Calling __get__(<__main__.MyDesc object at 0x7f1c9a1b5220>, None, <class '__main__.MyClass'>)


In [138]:
myobj.a     # invoke a.__get__(myobj, MyClass)

Calling __get__(<__main__.MyDesc object at 0x7f1c9a1b5220>, <Instance of MyClass>, <class '__main__.MyClass'>)


In [139]:
# MyClass.a = 5  # Overwrites the descriptor, so don't do this

In [140]:
myobj.a = 20   # invokes a.__set__(myobj, 20)

Calling __set__(<__main__.MyDesc object at 0x7f1c9a1b5220>, <Instance of MyClass>, 20)


In [141]:
del myobj.a    # invokes a.__delete__(myobj)

Calling __del__(<__main__.MyDesc object at 0x7f1c9a1b5220>, <Instance of MyClass>)


### Descriptor "trick": shared data

In [ ]:
class SharedValue:
    
    def __init__(self, value):
        self._value = value
        
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return self._value
    
    def __set__(self, obj, value):
        self._value = value
        
        
class NonsharedValue:
    def __init__(self, value):
        self._default_value = value
        self._values = {}
        
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return self._values.get(id(obj), self._default_value)
    
    def __set__(self, obj, value):
        self._values[id(obj)] = value

In [ ]:
class Shared:
    a = SharedValue(123)
    b = SharedValue(456)
    c = NonsharedValue(789)
    
    def __repr__(self):
        return f'<Shared a={self.a} b={self.b} c={self.c}>'

In [ ]:
s0 = Shared()
s1 = Shared()
print(s0, s1)

In [ ]:
s0.a = 'New avalue'  # ==> Shared.__set__(self, s0, 'New avalue')
s0.c = 'New cvalue'  # ==> NonShared.__set__(self, s0, 'New cvalue')
print(s0, s1)

Let's re-implement `@property`:

In [143]:
class myproperty:

    def __init__(self, getter, setter=None, deleter=None):
        self._getter = getter
        self._setter = setter
        self._deleter = deleter
        
    def __get__(self, inst, cls):
        print('Calling __get__')
        if inst is None:
            return self
        return self._getter(inst)
    
    def __set__(self, inst, value):
        print('Calling __set__')
        if self._setter is None:
            raise TypeError('value is read-only')
        self._setter(inst, value)
        
    def __delete__(self, inst):
        print('Calling __delete__')
        if self._deleter is None:
            raise TypeError('value is undeleteable')
        self._deleter(inst)
        
    def setter(self, setter):
        """Decorator to add a setter"""
        self._setter = setter
        return self
    
    def deleter(self, deleter):
        """Decorator to add a deleter"""
        self._deleter = deleter
        return self

In [144]:
class Foo:   # Foo()    Foo(object)
    
    @myproperty
    def bar(self):
        print('Calling getter for bar')
        return 'barval'
    #bar = myproperty(bar)
    
    @bar.setter
    def bar(self, value):
        print('Calling setter for bar')
        
#     _tmp0 = bar.setter
#     def bar(self, value):
#         print('Calling the setter for bar')
#     bar = _tmp0(bar)
        
    @bar.deleter
    def bar(self):
        print('Calling deleter for bar')
        
foo = Foo()

In [145]:
Foo.__dict__['bar']

In [146]:
Foo.bar

Calling __get__


In [147]:
foo.bar

Calling __get__
Calling getter for bar


'barval'

In [148]:
foo.bar = 10

Calling __set__
Calling setter for bar


In [149]:
foo.bar

Calling __get__
Calling getter for bar


'barval'

In [153]:
del foo.bar

Calling __delete__
Calling deleter for bar


In [154]:
Foo.bar

Calling __get__


In [155]:
property??

## Descriptor types

- A **data descriptor** is a descriptor that defines both `__get__` and `__set__`
- A **non-data descriptor** is a descriptor that defines only `__get__`

> Data and non-data descriptors differ in how overrides are calculated with respect to entries in an instance’s dictionary. If an instance’s dictionary has an entry with the same name as a data descriptor, the data descriptor takes precedence. If an instance’s dictionary has an entry with the same name as a non-data descriptor, the dictionary entry takes precedence.

So order of precedence in attribute access is:

- data descriptor
- instance `__dict__`
- non-data descriptor

In [156]:
class Foo(): 
    pass

foo = Foo()
foo.a = 5

In [157]:
foo.__dict__

{'a': 5}

In [158]:
class DataDescriptor():
    
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return 'data descriptor value'
    
    def __set__(self, obj, value):
        # Just make it a read-only data descriptor
        raise TypeError('read-only property')

In [159]:
class NonDataDescriptor():
    
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return 'non-data descriptor value'
        

In [160]:
class MyClass():
    data = DataDescriptor()
    nondata = NonDataDescriptor()

In [161]:
obj = MyClass()
obj.__dict__.update(
    data='instance data',
    nondata='instance nondata'
)

Data descriptors have precedence over instance data:

In [162]:
obj.data

'data descriptor value'

Instance data has precendence over non-data descriptors:

In [163]:
obj.nondata

'instance nondata'

In [164]:
obj.nondata = 'something else'

In [165]:
obj.nondata

'something else'

If we delete it from the instance dict, however, it _will_ invoke the non-data descriptor's `__get__` method

In [166]:
del obj.nondata

In [167]:
obj.nondata

'non-data descriptor value'

## Descriptor use case: cached property

In [168]:
class cached_property:  # "reify" is another name for this
    
    def __init__(self, getter):
        self._getter = getter
        self._name = None
        
    def __set_name__(self, cls, name):
        self._name = name
    
    def __get__(self, inst, cls):
        if inst is None:
            return self
        value = self._getter(inst)
        setattr(inst, self._name, value) # put the value in the instance __dict__
        # alternatively, inst.__dict__[name] = value
        return value
    
#     def __set__(self, obj, value):
#         obj.__dict__[self._name] = value

In [169]:
class CachedExample:
    
    @cached_property
    def prop(self):
        print('Calculating CachedExample.prop')
        return 42


In [170]:
ce = CachedExample()

In [171]:
ce.prop # adds prop to ce.__dict__

Calculating CachedExample.prop


42

In [172]:
ce.prop  # Since prop is non-data descriptor, it is not even accessed here

42

In [173]:
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)

In "real life", this is implemented in `functools.cached_property` (in Python 3.8+)

```python
import functools
help(functools.cached_property)
```

In [174]:
import functools
help(functools.cached_property)

Help on class cached_property in module functools:

class cached_property(builtins.object)
 |  cached_property(func)
 |  
 |  Methods defined here:
 |  
 |  __get__(self, instance, owner=None)
 |  
 |  __init__(self, func)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __set_name__(self, owner, name)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [175]:
functools.cached_property??

Open [Advanced OOP Lab](./advanced-oop-lab.ipynb)